In [3]:
import os
import glob

In [4]:
BASE_DIR = "/home/sliu/Desktop/Development/pytorch-dynamic-tsn/"
SPLITS_DIR = os.path.join(BASE_DIR, "data/splits/")
AGG_DATA_DIR = os.path.join(BASE_DIR, "data/preprocessed/AGG01/")

In [5]:
class_folders = filter(os.path.isdir, glob.glob(AGG_DATA_DIR + '/*'))

metadata = {}
listpaths = []
listlabels = []
for label, class_folder in enumerate(class_folders):
    metadata[label] = os.path.basename(class_folder)
    files = glob.glob(class_folder + '/*')
    labels = [label] * len(files)
    listpaths.extend(files)
    listlabels.extend(labels)

In [67]:
!pip3 install sklearn

  Using cached https://files.pythonhosted.org/packages/85/04/49633f490f726da6e454fddc8e938bbb5bfed2001681118d3814c219b723/scikit_learn-0.21.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/87/2d/e4656149cbadd3a8a0369fcd1a9c7d61cc7b87b3903b85389c70c989a696/numpy-1.16.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/72/4c/5f81e7264b0a7a8bd570810f48cd346ba36faedbd2ba255c873ad556de76/scipy-1.3.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/cd/c1/50a758e8247561e58cb87305b1e90b171b8c767b15b12a1734001f41d356/joblib-0.13.2-py2.py3-none-any.whl
  Running setup.py bdist_wheel for sklearn ... done
  Stored in directory: /home/sliu/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [6]:
import numpy as np
listpaths = np.array(listpaths)
listlabels = np.array(listlabels)

listpaths.shape, listlabels.shape

((23720,), (23720,))

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit

def _train_test_path_labels(all_paths, all_labels, test_size=0.1, random_state=42):
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size)
    train_idx, test_idx = next(iter(sss.split(all_paths, all_labels)))
    return all_paths[train_idx], all_paths[test_idx], all_labels[train_idx], all_labels[test_idx] 

def _skf_path_labels(all_paths, all_labels, outdir, out_prefix="", n_splits=5):
    # stratify dataset
    skf = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.2)
    for i, (train_idx, test_idx) in enumerate(skf.split(all_paths, all_labels)):
        X_train = all_paths[train_idx] # X_train is list of train data path 
        y_train = all_labels[train_idx] # y_train is list of label values

        # path for text of train path list
        train_prefix = "{}_train_split_{}.txt".format(out_prefix, i)
        train_filepath = os.path.join(outdir, train_prefix) 

        with open(train_filepath, 'w') as fp:
            for filepath, label in zip(X_train, y_train):
                fp.write("{} {}\n".format(filepath, label))
        
        X_test = all_paths[test_idx] # X_test is list of train data path
        y_test = all_labels[test_idx] # y_test is list of train data path

        # path for text of test path list
        test_prefix = "{}_val_split_{}.txt".format(out_prefix, i)
        test_filepath = os.path.join(outdir, test_prefix) 

        with open(test_filepath, 'w') as fp:
            for filepath, label in zip(X_test, y_test):
                fp.write("{} {}\n".format(filepath, label))

In [10]:
path_X_train, path_X_test, y_train, y_test = _train_test_path_labels(listpaths, listlabels)

In [18]:
_skf_path_labels(path_X_train, y_train, SPLITS_DIR, out_prefix="aggresion")

In [20]:
test_filename = "aggression_test_split_0.txt"
test_filepath = os.path.join(SPLITS_DIR, test_filename)

with open(test_filepath, 'w') as fp:
    for filepath, label in zip(path_X_train, y_test):
        fp.write("{} {}\n".format(filepath, label))

0